In [22]:
using DataFrames, Feather

In [88]:
@enum FailureType begin
    # General ones
    Unknown
    InferredFailure
    PrintingChange
    TestAmbiguities
    NewAmbiguity
    ApproxError
    DocTest
    NewExceptionType
    DownloadError
    UnexpectedPass
    BadBoy
    # Specific
    InferenceStackOverFlow
    IRTools
    LLVMModuleIR
    BoundsErrorCompiler
    Cassette
    GPUErrors
end

In [24]:
if !isdir("data")
    run(`tar -xvf data.tar.gz`)
end
primary  = Feather.read("data/primary.feather");
against = Feather.read("data/against.feather");

data/
data/primary.feather
data/primary.json
data/against.feather
data/against.json


In [25]:
package_results = join(primary, against,
     on=:uuid, kind=:left, makeunique=true, indicator=:source);

In [26]:
changed_tests = filter(test->test.source == "both" &&
                 test.status != test.status_1, package_results)
fails = changed_tests[changed_tests[!, :status] .== ":fail", :];
sort!(fails , :name);
insert!(fails, size(fails, 2), fill(Unknown, size(fails, 1)), :why);

In [27]:
query(fails, s) = filter(row -> occursin(s, row[:log]), fails)    

function update_reason!(fails, needle, why)
    idxs = findall(row -> occursin(needle, row), fails.log)
    fails.why[idxs] .= why
    return fails
end

total_unknown(fails) = count(x -> x.why == Unknown, eachrow(fails))

total_unknown (generic function with 1 method)

In [89]:
# Generic ones
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "detect_ambiguities(", TestAmbiguities)

update_reason!(fails, "Expression: all_doctests()", DocTest)
update_reason!(fails, "does not match inferred return type", InferredFailure);
update_reason!(fails, "Expression: isapprox", ApproxError)
update_reason!(fails, "is ambiguous. Candidates:", NewAmbiguity)
update_reason!(fails, "Evaluated: \"", PrintingChange)
update_reason!(fails, "Log Test Failed at", PrintingChange)
update_reason!(fails, "Expression: occursin(r\"", PrintingChange)
update_reason!(fails, "Expression: startswith(", PrintingChange)
update_reason!(fails, r"Expression: (\N.*?) == ", PrintingChange)
update_reason!(fails, "Evaluated: occursin(", PrintingChange)
update_reason!(fails, "Evaluated: endswith(", PrintingChange)
update_reason!(fails, "- DIFF ------------------------", PrintingChange)

update_reason!(fails, "      Thrown: ", NewExceptionType)
update_reason!(fails, "The requested URL returned error", DownloadError)
update_reason!(fails, "gzip: stdin: not in gzip format", DownloadError)
update_reason!(fails, "Unexpected Pass", UnexpectedPass)

# Specific ones for this release
update_reason!(fails, "Params not defined", IRTools);
update_reason!(fails, "Core.Compiler.IRCode(", IRTools);
update_reason!(fails, "/.julia/packages/IRTools/", IRTools);
update_reason!(fails, "Module IR does not contain specified entry function", LLVMModuleIR);
update_reason!(fails, "Internal error: encountered unexpected error in runtime:\nBoundsError", BoundsErrorCompiler
);
update_reason!(fails, "@ Cassette", Cassette)
update_reason!(fails, "Test.detect_ambiguities", TestAmbiguities)
update_reason!(fails, "unsupported or misplaced expression \"return\" in function", GPUErrors)
update_reason!(fails, "intersect_aside at /workspace/srcdir/src/subtype.c:2146", InferenceStackOverFlow);

# Baddies

# https://github.com/JuliaDiffusionBayes/DiffusionDefinition.jl/issues/11
update_reason!(fails, "DD.remove_curly(Vector{Float64}) == Array", BadBoy)
update_reason!(fails, "Unknown observable eltype", BadBoy)
update_reason!(fails, "ArgumentError: Invalid type: typename(SwingEq)", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/Tricks/Omjx9/test/runtests.jl:89", BadBoy)
update_reason!(fails, "Judycon ~/.julia/packages/Judycon/Zc2IV/src/Judycon.jl:7", BadBoy)
update_reason!(fails, ": Error During Test at /home/pkgeval/.julia/packages/LOLTools/K3Wzy/test/loltools/championmasteryv4.jl:1", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/Libtask/Zo6uM/test/ctask.jl:23", BadBoy)
update_reason!(fails, "~/.julia/packages/MonthlyDates/vCogo/src/MonthlyDates.jl:72", BadBoy)
update_reason!(fails, "type Method has no field ambig", BadBoy)
update_reason!(fails, "oadError: UndefVarError: LAST_SHOWN_LINE_INFOS not definedfatal: ", BadBoy)
update_reason!(fails, "/home/pkgeval/.julia/packages/CQLdriver/QjN09/deps/build.jl:1", BadBoy)
update_reason!(fails, "Cannot `convert` an object of type ForneyLab.Variable to an object of type Symbol", BadBoy)
update_reason!(fails, "est Failed at /home/pkgeval/.julia/packages/XPORTA/QaHQb/test/integration/filesystem.jl:41", BadBoy)
update_reason!(fails, "During Test at /home/pkgeval/.julia/packages/AlignedBinaryFormat/Nlf6L/test/runtests.jl:59", BadBoy)
update_reason!(fails, "ApproximateComputations/fS6jW/src/ASTReplacementApproximation.jl:121", BadBoy)


nothing

In [90]:
total_unknown(fails)

47

In [86]:
filter(x -> x.why == Unknown, fails).name[15:30]

16-element Array{String,1}:
 "GeoGreensFunctions"
 "GslibIO"
 "HAML"
 "InfiniteLinearAlgebra"
 "InstantiateFromURL"
 "InterProcessCommunication"
 "JuliennedArrays"
 "LLVM"
 "MCMCChains"
 "MLLabelUtils"
 "Memento"
 "NLopt"
 "OhMyREPL"
 "PhaseSpaceIO"
 "PhysicsInformedML"
 "ProximalOperators"

In [87]:
z = []
for i in instances(FailureType)
    n = count(row -> row.why == i, eachrow(fails))
    push!(z, (n, i))
end
sort!(z; rev=true)

19-element Array{Any,1}:
 (84, PrintingChange)
 (47, Unknown)
 (39, IRTools)
 (28, LLVMModuleIR)
 (26, TestAmbiguities)
 (17, DocTest)
 (14, GPUErrors)
 (13, Cassette)
 (13, InferenceStackOverFlow)
 (12, BadBoy)
 (9, NewAmbiguity)
 (8, BoundsErrorCompiler)
 (4, NewExceptionType)
 (2, UnexpectedPass)
 (2, DownloadError)
 (2, ApproxError)
 (2, InferredFailure)
 (0, OldMbedTLSSyntax)
 (0, ParameterUpperBound)

In [83]:
query(fails, "- DIFF ------------------------")

,julia,name,uuid
,String,String,String
1,"v""1.6.0-DEV-3d1de029bc""",ImageInTerminal,"UUID(""d8c32880-2388-543b-8c61-d9f865259254"")"
2,"v""1.6.0-DEV-3d1de029bc""",MLDataPattern,"UUID(""9920b226-0b2a-5f5f-9153-9aa70a013f8b"")"
3,"v""1.6.0-DEV-3d1de029bc""",ReferenceTests,"UUID(""324d217c-45ce-50fc-942e-d289b448e8cf"")"
